# 🔍 Data Quality Analysis & Cleaning
## Global Weather Repository - Data Integrity Check (Did because i found a typos when doing the forecasting)

---

**Objectives:**
1. Analyze missing values and data gaps
2. Identify country name typos and inconsistencies (e.g., 'inde' → 'India')
3. Find other data quality issues
4. Create data cleaning recommendations

---

## 1. Setup & Load Raw Data

In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, get_close_matches
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")

✅ Libraries loaded


In [2]:
# Load RAW data (before any processing)
RAW_PATH = '../data/raw/GlobalWeatherRepository.csv'
df_raw = pd.read_csv(RAW_PATH)

print(f"📊 Raw Dataset: {len(df_raw):,} rows × {len(df_raw.columns)} columns")
print(f"🌍 Unique countries: {df_raw['country'].nunique()}")
print(f"📍 Unique locations: {df_raw['location_name'].nunique()}")

📊 Raw Dataset: 114,203 rows × 41 columns
🌍 Unique countries: 211
📍 Unique locations: 255


---
## 2. Country Name Analysis

In [3]:
# Get all unique country names
countries = df_raw['country'].unique()
country_counts = df_raw['country'].value_counts()

print(f"📊 Total unique country values: {len(countries)}")
print(f"\n🔤 All country names (sorted alphabetically):")
for c in sorted(countries):
    print(f"   {c}: {country_counts[c]:,} records")

📊 Total unique country values: 211

🔤 All country names (sorted alphabetically):
   Afghanistan: 587 records
   Albania: 587 records
   Algeria: 586 records
   Andorra: 587 records
   Angola: 587 records
   Antigua and Barbuda: 583 records
   Argentina: 585 records
   Armenia: 587 records
   Australia: 587 records
   Austria: 586 records
   Azerbaijan: 586 records
   Bahamas: 581 records
   Bahrain: 587 records
   Bangladesh: 585 records
   Barbados: 584 records
   Belarus: 587 records
   Belgium: 1,137 records
   Belize: 583 records
   Benin: 586 records
   Bhutan: 586 records
   Bolivia: 1,165 records
   Bosnia and Herzegovina: 587 records
   Botswana: 587 records
   Brazil: 583 records
   Brunei Darussalam: 586 records
   Bulgaria: 1,327 records
   Burkina Faso: 586 records
   Burundi: 587 records
   Bélgica: 1 records
   Cambodia: 586 records
   Cameroon: 585 records
   Canada: 584 records
   Cape Verde: 587 records
   Central African Republic: 586 records
   Chad: 587 records
   C

In [4]:
# Known correct country names (reference list)
KNOWN_COUNTRIES = [
    'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda',
    'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
    'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
    'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria',
    'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
    'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros',
    'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Czechia',
    'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador',
    'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini',
    'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
    'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
    'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq',
    'Ireland', 'Israel', 'Italy', 'Ivory Coast', 'Jamaica', 'Japan', 'Jordan',
    'Kazakhstan', 'Kenya', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia',
    'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg',
    'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands',
    'Mauritania', 'Mauritius', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia',
    'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal',
    'Netherlands', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Korea',
    'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine', 'Panama',
    'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal',
    'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia',
    'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe',
    'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore',
    'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Korea',
    'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland',
    'Syria', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 'Togo', 'Tonga',
    'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Tuvalu', 'Uganda',
    'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States of America',
    'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City', 'Venezuela', 'Vietnam',
    'Yemen', 'Zambia', 'Zimbabwe'
]

print(f"📚 Reference list: {len(KNOWN_COUNTRIES)} known countries")

📚 Reference list: 197 known countries


In [5]:
def find_similar_countries(country_name, reference_list, threshold=0.6):
    """
    Find similar country names using string similarity.
    """
    matches = []
    country_lower = country_name.lower().strip()
    
    for ref in reference_list:
        ref_lower = ref.lower().strip()
        
        # Exact match (case insensitive)
        if country_lower == ref_lower:
            return [{'match': ref, 'score': 1.0, 'type': 'exact'}]
        
        # Similarity score
        score = SequenceMatcher(None, country_lower, ref_lower).ratio()
        if score >= threshold:
            matches.append({'match': ref, 'score': score, 'type': 'similar'})
    
    return sorted(matches, key=lambda x: x['score'], reverse=True)[:3]

# Find potential typos and inconsistencies
print("🔍 ANALYZING COUNTRY NAMES FOR TYPOS AND INCONSISTENCIES...")
print("="*70)

issues = []
for country in sorted(countries):
    matches = find_similar_countries(country, KNOWN_COUNTRIES)
    
    if not matches:
        # No match found - might be a typo or unknown country
        issues.append({
            'original': country,
            'records': country_counts[country],
            'issue': 'No match found',
            'suggestion': 'Manual review needed'
        })
    elif matches[0]['type'] != 'exact' and matches[0]['score'] < 1.0:
        # Similar but not exact - potential typo
        issues.append({
            'original': country,
            'records': country_counts[country],
            'issue': f"Similar to '{matches[0]['match']}'",
            'suggestion': matches[0]['match'],
            'score': matches[0]['score']
        })

issues_df = pd.DataFrame(issues)
if len(issues_df) > 0:
    print(f"\n⚠️ Found {len(issues_df)} potential issues:")
    print(issues_df.to_string(index=False))
else:
    print("\n✅ No obvious typos found")

🔍 ANALYZING COUNTRY NAMES FOR TYPOS AND INCONSISTENCIES...

⚠️ Found 31 potential issues:
                        original  records                                         issue                       suggestion    score
               Brunei Darussalam      586                                No match found             Manual review needed      NaN
                         Bélgica        1                         Similar to 'Bulgaria'                         Bulgaria 0.666667
                   Cote d'Ivoire      126                                No match found             Manual review needed      NaN
    Democratic Republic of Congo      584                                No match found             Manual review needed      NaN
                         Estonie        1                          Similar to 'Estonia'                          Estonia 0.857143
                    Fiji Islands      587                          Similar to 'Finland'                          Finland 0.631579


In [6]:
# Deep search for specific patterns
# IMPORTANT: Only match EXACT patterns (not substrings) to avoid false positives
print("\n🔍 DEEP SEARCH FOR COMMON TYPO PATTERNS...")
print("="*70)

# Exact match patterns only (to avoid 'uk' matching 'Ukraine')
EXACT_TYPO_PATTERNS = {
    'inde': 'India',
    'usa': 'United States of America',
    'u.s.a.': 'United States of America',
    'u.s.a': 'United States of America',
    'u.k.': 'United Kingdom',
    'u.k': 'United Kingdom',
    'uae': 'United Arab Emirates',
    'u.a.e.': 'United Arab Emirates',
    'brasil': 'Brazil',
    'rusia': 'Russia',
    'russie': 'Russia',
    'chine': 'China',
    'japon': 'Japan',
    'allemagne': 'Germany',
    'espagne': 'Spain',
    'italie': 'Italy',
    'mexique': 'Mexico',
    'maroc': 'Morocco',
    'egypte': 'Egypt',
    'turquie': 'Turkey',
    'grece': 'Greece',
    'suisse': 'Switzerland',
    'autriche': 'Austria',
    'belgique': 'Belgium',
    'pologne': 'Poland',
    'coree': 'South Korea',
    'argentine': 'Argentina',
    'royaume-uni': 'United Kingdom',
    'royaume uni': 'United Kingdom',
    'etats-unis': 'United States of America',
    'etats unis': 'United States of America',
}

found_patterns = []
for country in countries:
    country_lower = country.lower().strip()
    
    # Only check for EXACT matches (not substrings)
    if country_lower in EXACT_TYPO_PATTERNS:
        found_patterns.append({
            'found': country,
            'pattern': country_lower,
            'should_be': EXACT_TYPO_PATTERNS[country_lower],
            'records': country_counts[country]
        })

# Check for case issues (but skip valid countries)
for country in countries:
    # Skip if it's a known valid country
    if country in KNOWN_COUNTRIES:
        continue
    
    # Check if lowercase version matches a known country
    country_lower = country.lower().strip()
    for known in KNOWN_COUNTRIES:
        if country_lower == known.lower() and country != known:
            found_patterns.append({
                'found': country,
                'pattern': 'case_issue',
                'should_be': known,
                'records': country_counts[country]
            })
            break

if found_patterns:
    patterns_df = pd.DataFrame(found_patterns)
    print(f"\n⚠️ Found {len(patterns_df)} pattern matches:")
    print(patterns_df.to_string(index=False))
else:
    print("\n✅ No common typo patterns found")


🔍 DEEP SEARCH FOR COMMON TYPO PATTERNS...

⚠️ Found 2 pattern matches:
  found pattern should_be  records
   Inde    inde     India        1
Mexique mexique    Mexico        1


In [7]:
# Check for near-duplicate country names
print("\n🔍 CHECKING FOR NEAR-DUPLICATE COUNTRY NAMES...")
print("="*70)

duplicates = []
countries_list = list(countries)

for i, c1 in enumerate(countries_list):
    for c2 in countries_list[i+1:]:
        # Skip if same
        if c1 == c2:
            continue
        
        # Check similarity
        score = SequenceMatcher(None, c1.lower(), c2.lower()).ratio()
        
        if score >= 0.85:  # Very similar (increased threshold)
            duplicates.append({
                'country_1': c1,
                'records_1': country_counts[c1],
                'country_2': c2,
                'records_2': country_counts[c2],
                'similarity': f"{score:.1%}"
            })

if duplicates:
    dup_df = pd.DataFrame(duplicates)
    print(f"\n⚠️ Found {len(dup_df)} potential duplicates:")
    print(dup_df.to_string(index=False))
else:
    print("\n✅ No near-duplicate country names found")


🔍 CHECKING FOR NEAR-DUPLICATE COUNTRY NAMES...

⚠️ Found 7 potential duplicates:
               country_1  records_1                    country_2  records_2 similarity
               Australia        587                      Austria        586      87.5%
                 Estonia        586                      Estonie          1      85.7%
                 Estonia        586                      Letonia          1      85.7%
                 Iceland        586                      Ireland        585      85.7%
            Saudi Arabia        585                Saudi Arabien          1      88.0%
            Turkmenistan        586                 Turkménistan          1      91.7%
United States of America        581 USA United States of America          3      92.3%


In [8]:
# Check for countries with very few records (potential typos)
print("\n🔍 COUNTRIES WITH VERY FEW RECORDS (< 100):")
print("="*70)

low_record_countries = country_counts[country_counts < 100].sort_values()

if len(low_record_countries) > 0:
    print(f"\n⚠️ Found {len(low_record_countries)} countries with < 100 records:")
    for country, count in low_record_countries.items():
        # Find possible correction
        matches = find_similar_countries(country, KNOWN_COUNTRIES, threshold=0.5)
        if matches and matches[0]['type'] == 'exact':
            suggestion = '✓ Valid (low data)'
        elif matches:
            suggestion = f"Maybe: {matches[0]['match']}"
        else:
            suggestion = 'Unknown'
        print(f"   {country}: {count} records → {suggestion}")
else:
    print("\n✅ All countries have >= 100 records")


🔍 COUNTRIES WITH VERY FEW RECORDS (< 100):

⚠️ Found 25 countries with < 100 records:
   Lao People's Democratic Republic: 1 records → Maybe: Dominican Republic
   Libya: 1 records → ✓ Valid (low data)
   Kosovo: 1 records → Unknown
   Togo: 1 records → ✓ Valid (low data)
   Komoren: 1 records → Maybe: Comoros
   Malásia: 1 records → Maybe: Malaysia
   كولومبيا: 1 records → Unknown
   Colombia: 1 records → ✓ Valid (low data)
   Inde: 1 records → Maybe: India
   Estonie: 1 records → Maybe: Estonia
   Гватемала: 1 records → Unknown
   Letonia: 1 records → Maybe: Estonia
   Polônia: 1 records → Maybe: Poland
   Mexique: 1 records → Maybe: Mexico
   Marrocos: 1 records → Maybe: Morocco
   Польша: 1 records → Unknown
   Jemen: 1 records → Maybe: Yemen
   Турция: 1 records → Unknown
   Saint-Vincent-et-les-Grenadines: 1 records → Maybe: Saint Vincent and the Grenadines
   Südkorea: 1 records → Maybe: South Korea
   Saudi Arabien: 1 records → Maybe: Saudi Arabia
   Bélgica: 1 records → Maybe

---
## 3. Location Name Analysis

In [9]:
# Analyze location names for issues
print("🔍 ANALYZING LOCATION NAMES...")
print("="*70)

locations = df_raw['location_name'].value_counts()

# Check for duplicate location names in different countries (might indicate issues)
location_by_country = df_raw.groupby('location_name')['country'].nunique()
multi_country_locations = location_by_country[location_by_country > 1]

print(f"\n📍 Locations appearing in multiple countries: {len(multi_country_locations)}")
if len(multi_country_locations) > 0:
    print("\nTop 10 multi-country locations:")
    for loc, count in multi_country_locations.sort_values(ascending=False).head(10).items():
        countries_for_loc = df_raw[df_raw['location_name'] == loc]['country'].unique()
        print(f"   {loc}: {count} countries → {', '.join(countries_for_loc[:5])}...")

🔍 ANALYZING LOCATION NAMES...

📍 Locations appearing in multiple countries: 12

Top 10 multi-country locations:
   Moroni: 3 countries → Comoros, USA United States of America, Komoren...
   Beirut: 2 countries → Lebanon, Польша...
   Bogot: 2 countries → Bulgaria, Hungary...
   Bern: 2 countries → Switzerland, Bélgica...
   Kingstown: 2 countries → Saint Vincent and the Grenadines, Saint-Vincent-et-les-Grenadines...
   Lom: 2 countries → Bulgaria, Russia...
   New Delhi: 2 countries → India, Inde...
   Riga: 2 countries → Latvia, Letonia...
   Sanaa: 2 countries → Yemen, Jemen...
   Seoul: 2 countries → South Korea, Südkorea...


In [10]:
# Check for location names that might be country names (data entry errors)
print("\n🔍 CHECKING IF ANY LOCATION NAMES ARE ACTUALLY COUNTRY NAMES...")
print("="*70)

suspicious_locations = []
for loc in df_raw['location_name'].unique():
    matches = find_similar_countries(loc, KNOWN_COUNTRIES, threshold=0.9)
    if matches and matches[0]['score'] >= 0.9:
        # Check if this location is in the same country
        loc_country = df_raw[df_raw['location_name'] == loc]['country'].iloc[0]
        if matches[0]['match'].lower() != loc_country.lower():
            suspicious_locations.append({
                'location': loc,
                'in_country': loc_country,
                'looks_like_country': matches[0]['match'],
                'records': locations[loc]
            })

if suspicious_locations:
    sus_df = pd.DataFrame(suspicious_locations)
    print(f"\n⚠️ Found {len(sus_df)} suspicious locations:")
    print(sus_df.to_string(index=False))
else:
    print("\n✅ No suspicious location names found")


🔍 CHECKING IF ANY LOCATION NAMES ARE ACTUALLY COUNTRY NAMES...

⚠️ Found 5 suspicious locations:
  location                   in_country looks_like_country  records
      Laos                    Indonesia               Laos      586
   Grenada USA United States of America            Grenada        1
     Palau USA United States of America              Palau        1
Costa Rica                     كولومبيا         Costa Rica        1
   Moldova                      Polônia            Moldova        1


---
## 4. Missing Values Analysis

In [11]:
# Comprehensive missing values analysis
print("🔍 MISSING VALUES ANALYSIS")
print("="*70)

missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)

missing_df = pd.DataFrame({
    'column': missing.index,
    'missing_count': missing.values,
    'missing_pct': missing_pct.values
}).sort_values('missing_pct', ascending=False)

print("\n📊 Missing Values by Column:")
print(missing_df[missing_df['missing_count'] > 0].to_string(index=False))

total_missing = missing.sum()
total_cells = len(df_raw) * len(df_raw.columns)
print(f"\n📊 Total missing cells: {total_missing:,} / {total_cells:,} ({total_missing/total_cells*100:.2f}%)")

🔍 MISSING VALUES ANALYSIS

📊 Missing Values by Column:
Empty DataFrame
Columns: [column, missing_count, missing_pct]
Index: []

📊 Total missing cells: 0 / 4,682,323 (0.00%)


In [12]:
# Visualize missing values
missing_cols = missing_df[missing_df['missing_count'] > 0]

if len(missing_cols) > 0:
    fig = px.bar(
        missing_cols,
        x='column',
        y='missing_pct',
        title='📊 Missing Values by Column (%)',
        color='missing_pct',
        color_continuous_scale='Reds'
    )
    fig.update_layout(xaxis_tickangle=-45, height=500)
    fig.show()
else:
    print("✅ No missing values!")

✅ No missing values!


In [13]:
# Missing values by country
print("\n🔍 MISSING VALUES BY COUNTRY")
print("="*70)

country_missing = df_raw.groupby('country').apply(
    lambda x: x.isnull().sum().sum() / (len(x) * len(x.columns)) * 100
).sort_values(ascending=False)

print("\n⚠️ Top 20 Countries with Most Missing Data:")
for country, pct in country_missing.head(20).items():
    print(f"   {country}: {pct:.2f}% missing")


🔍 MISSING VALUES BY COUNTRY

⚠️ Top 20 Countries with Most Missing Data:
   Afghanistan: 0.00% missing
   Albania: 0.00% missing
   Algeria: 0.00% missing
   Andorra: 0.00% missing
   Angola: 0.00% missing
   Antigua and Barbuda: 0.00% missing
   Argentina: 0.00% missing
   Armenia: 0.00% missing
   Australia: 0.00% missing
   Austria: 0.00% missing
   Azerbaijan: 0.00% missing
   Bahamas: 0.00% missing
   Bahrain: 0.00% missing
   Bangladesh: 0.00% missing
   Barbados: 0.00% missing
   Belarus: 0.00% missing
   Belgium: 0.00% missing
   Belize: 0.00% missing
   Benin: 0.00% missing
   Bhutan: 0.00% missing


In [14]:
# Visualize
top_missing = country_missing.head(30).reset_index()
top_missing.columns = ['country', 'missing_pct']

fig = px.bar(
    top_missing,
    x='country',
    y='missing_pct',
    title='📊 Top 30 Countries by Missing Data %',
    color='missing_pct',
    color_continuous_scale='Reds'
)
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

---
## 5. Data Sufficiency Analysis

In [15]:
# Analyze which countries have insufficient data for modeling
print("🔍 DATA SUFFICIENCY ANALYSIS")
print("="*70)

# Parse dates
df_raw['last_updated'] = pd.to_datetime(df_raw['last_updated'])
df_raw['date'] = df_raw['last_updated'].dt.date

# Calculate daily data points per country
country_daily = df_raw.groupby(['country', 'date']).size().reset_index(name='records_per_day')
country_days = country_daily.groupby('country').size().reset_index(name='total_days')
country_days = country_days.sort_values('total_days', ascending=True)

print(f"\n📅 Days of data per country:")
print(f"   Min: {country_days['total_days'].min()} days")
print(f"   Max: {country_days['total_days'].max()} days")
print(f"   Mean: {country_days['total_days'].mean():.1f} days")
print(f"   Median: {country_days['total_days'].median():.1f} days")

🔍 DATA SUFFICIENCY ANALYSIS

📅 Days of data per country:
   Min: 1 days
   Max: 586 days
   Mean: 513.0 days
   Median: 585.0 days


In [16]:
# Countries with insufficient data (< 20 days needed for lag features)
MIN_DAYS_REQUIRED = 20
insufficient = country_days[country_days['total_days'] < MIN_DAYS_REQUIRED]

print(f"\n⚠️ Countries with < {MIN_DAYS_REQUIRED} days of data ({len(insufficient)} countries):")
print(insufficient.to_string(index=False))


⚠️ Countries with < 20 days of data (25 countries):
                         country  total_days
                         Bélgica           1
                        Colombia           1
                         Estonie           1
                         Malásia           1
                           Jemen           1
                          Kosovo           1
                        Marrocos           1
                         Mexique           1
Lao People's Democratic Republic           1
                         Letonia           1
                           Libya           1
                            Inde           1
                         Komoren           1
                         Polônia           1
                   Saudi Arabien           1
 Saint-Vincent-et-les-Grenadines           1
                          Польша           1
                    Turkménistan           1
                            Togo           1
                        Südkorea           1
  

In [17]:
# Visualize data sufficiency
fig = px.histogram(
    country_days,
    x='total_days',
    nbins=30,
    title='📊 Distribution of Days of Data per Country',
    labels={'total_days': 'Days of Data', 'count': 'Number of Countries'}
)
fig.add_vline(x=MIN_DAYS_REQUIRED, line_dash='dash', line_color='red', 
              annotation_text=f'Min Required ({MIN_DAYS_REQUIRED} days)')
fig.update_traces(marker_color='#4ECDC4')
fig.update_layout(height=400)
fig.show()

In [18]:
# Bar chart of countries with least data
bottom_30 = country_days.head(30)

fig = px.bar(
    bottom_30,
    x='country',
    y='total_days',
    title='📊 30 Countries with Least Data',
    color='total_days',
    color_continuous_scale='RdYlGn'
)
fig.add_hline(y=MIN_DAYS_REQUIRED, line_dash='dash', line_color='red',
              annotation_text=f'Min Required')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

---
## 6. Data Quality Summary & Recommendations

In [19]:
# Generate comprehensive summary
print("="*70)
print("📊 DATA QUALITY SUMMARY")
print("="*70)

print(f"\n1️⃣ DATASET OVERVIEW:")
print(f"   Total records: {len(df_raw):,}")
print(f"   Total countries: {df_raw['country'].nunique()}")
print(f"   Total locations: {df_raw['location_name'].nunique()}")
print(f"   Date range: {df_raw['date'].min()} to {df_raw['date'].max()}")

print(f"\n2️⃣ MISSING VALUES:")
cols_with_missing = len(missing_df[missing_df['missing_count'] > 0])
print(f"   Columns with missing values: {cols_with_missing}")
print(f"   Overall missing rate: {total_missing/total_cells*100:.2f}%")

print(f"\n3️⃣ DATA SUFFICIENCY:")
sufficient = len(country_days) - len(insufficient)
print(f"   Countries with sufficient data (≥{MIN_DAYS_REQUIRED} days): {sufficient}")
print(f"   Countries with insufficient data: {len(insufficient)}")

print(f"\n4️⃣ POTENTIAL DATA QUALITY ISSUES:")
if len(issues_df) > 0:
    print(f"   Country name issues found: {len(issues_df)}")
else:
    print(f"   Country name issues: None found")

if len(low_record_countries) > 0:
    print(f"   Countries with < 100 records: {len(low_record_countries)}")

📊 DATA QUALITY SUMMARY

1️⃣ DATASET OVERVIEW:
   Total records: 114,203
   Total countries: 211
   Total locations: 255
   Date range: 2024-05-16 to 2025-12-24

2️⃣ MISSING VALUES:
   Columns with missing values: 0
   Overall missing rate: 0.00%

3️⃣ DATA SUFFICIENCY:
   Countries with sufficient data (≥20 days): 186
   Countries with insufficient data: 25

4️⃣ POTENTIAL DATA QUALITY ISSUES:
   Country name issues found: 31
   Countries with < 100 records: 25


In [20]:
# Create cleaning recommendations
print("\n" + "="*70)
print("🔧 RECOMMENDED DATA CLEANING ACTIONS")
print("="*70)

recommendations = []

# Add country name fixes from similarity analysis
if len(issues_df) > 0:
    for _, row in issues_df.iterrows():
        if 'suggestion' in row and row['suggestion'] != 'Manual review needed':
            # Skip if suggestion equals original (valid country)
            if row['original'] != row.get('suggestion', ''):
                recommendations.append({
                    'action': 'Rename country',
                    'from': row['original'],
                    'to': row.get('suggestion', 'Review manually'),
                    'affected_records': row['records'],
                    'priority': 'High' if row['records'] > 100 else 'Medium'
                })

# Add pattern fixes (only exact typos)
if found_patterns:
    for p in found_patterns:
        # Avoid duplicates and false positives
        if p['found'] != p['should_be']:
            recommendations.append({
                'action': 'Fix typo/format',
                'from': p['found'],
                'to': p['should_be'],
                'affected_records': p['records'],
                'priority': 'High'
            })

if recommendations:
    rec_df = pd.DataFrame(recommendations).drop_duplicates(subset=['from', 'to'])
    print("\n📋 Recommended Changes:")
    print(rec_df.to_string(index=False))
    
    # Save recommendations
    rec_df.to_csv('../reports/data_cleaning_recommendations.csv', index=False)
    print("\n✅ Recommendations saved to reports/data_cleaning_recommendations.csv")
else:
    print("\n✅ No automatic fixes needed")


🔧 RECOMMENDED DATA CLEANING ACTIONS

📋 Recommended Changes:
        action                            from                               to  affected_records priority
Rename country                         Bélgica                         Bulgaria                 1   Medium
Rename country                         Estonie                          Estonia                 1   Medium
Rename country                    Fiji Islands                          Finland               587     High
Rename country                            Inde                            India                 1   Medium
Rename country                           Jemen                            Yemen                 1   Medium
Rename country                     Kyrghyzstan                       Kyrgyzstan               586     High
Rename country                         Letonia                          Estonia                 1   Medium
Rename country                       Macedonia                  North Macedonia    

In [21]:
# Save all analysis results
import os
os.makedirs('../reports', exist_ok=True)

# Save insufficient countries
insufficient.to_csv('../reports/insufficient_data_countries.csv', index=False)
print("✅ Insufficient data countries saved")

# Save missing values analysis
missing_df.to_csv('../reports/missing_values_analysis.csv', index=False)
print("✅ Missing values analysis saved")

# Save country data summary
country_days.to_csv('../reports/country_data_summary.csv', index=False)
print("✅ Country data summary saved")

print("\n📁 All reports saved to ../reports/")

✅ Insufficient data countries saved
✅ Missing values analysis saved
✅ Country data summary saved

📁 All reports saved to ../reports/


---
## 📊 Summary

This notebook analyzed:
1. **Country name typos** - Found potential misspellings like 'inde' → 'India'
2. **Near-duplicate countries** - Identified similar country names that might be duplicates
3. **Missing values** - Analyzed missing data by column and by country
4. **Data sufficiency** - Identified countries with insufficient data for modeling
5. **Generated recommendations** - Created actionable cleaning suggestions

### Next Steps:
1. Apply the recommended data cleaning fixes
2. Re-run the forecasting pipeline with cleaned data
3. Monitor for new data quality issues